<img style="float: left;" src="D2Klogo.png" width="20%"> 

<center>

# Welcome to Data Analytics #
### Introduction to your first Data project ###

### Code Demo for Day 1 ###

# This demo is a toy example of how to solve a data analytics problem: 
## Question $\rightarrow$ Data $\rightarrow$ Answer $\rightarrow$ More questions $\rightarrow$ ...


## Your ultimate goal is to walk through your own Data Science Pipeline by completing a real data project.

<br/>

## Question: Can we predict test scores for students?

How can we answer this question?

<br/>


## Ideas:

1. which test scores are we predicting? 
2. What features can we use to predict test scores?
3. What kind of model can we use to predict test scores?
4. What are the most important features to predict test scores?
5. Are there features that interact with each other that affect test scores?


<br/>


##About Dataset 

This dataset includes scores from three test scores of students at a (fictional) public school and a variety of personal and socio-economic factors that may have interaction effects upon them.

#  **Predicting Student Test Scores**

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import plotly as px
import matplotlib as plt
import seaborn as sns

**In this notebook:**
1. Exploratory Data Analysis
2. Preprocessing
3. Model Creation
4. Model Analysis
5. Conclusion

In [ ]:
df = pd.read_excel('/content/TestScoresDataset.xlsx')
df.head(5) #print the first 5 rows

In [ ]:
df.columns #print columns in the dataframe

*Given in the dataset description, we have a data dictionary we will refer back to:*


TestPrep: test preparation course (completed or none)


PracticeSport: how often the student parctice sport

IsFirstChild: if the child is first child in the family or not

NrSiblings : Number of siblings the student has

TransportMeans: Means of transport to school

WklyStudyHours: Weekly self-study hours

MathScore: math test score

ReadingScore: reading test score

WritingScore writing test score

In [ ]:
df.shape

In [ ]:
#we have an unnecessary column, 'Unamed: 0', which seems to be an index column. 
#we will remove this column, as it provides no utility at this moment, or for the 
#model we will create later

df = df.drop('Unnamed: 0', axis=1)
df.head(5) #shows the column has been dropped

In [ ]:
#I want to return the datatypes of each feature:
df.dtypes

In [ ]:
#all of our features are objects except NrSibligns
#MathScore, ReadingScore, WritingScore are our target values

#lets summarize the dataset (the numeric values)
df.describe()

In [ ]:
#Weekly study hours currently is in an improper format
#Lets fix this

#first, lets view the unique weekly study hours values:
df['WklyStudyHours'].unique()

In [ ]:
#let's remove these rows
#we also will assign this new data to a new dataframe
df2 = df.dropna(subset=['WklyStudyHours']).copy()
df2['WklyStudyHours'].unique() #returns unique values after dropping 'nan'

In [ ]:
df2.head()

In [ ]:
#I see multiple 'NaN' values outside of this feature
#We need to decide how to handle these rows

#they aren't numeric values, so we cannot substitute them with the means
#we will fix this after the EDA (exploratory data analysis)

**Exploratory Data Analysis**

In [ ]:
import matplotlib.pyplot as plt

*TestPrep Distribution*

In [ ]:
#testprep distribution
plt.figure(figsize=(6,4))
df2['TestPrep'].value_counts().plot(kind='bar')
plt.xlabel('Test Preparation Course')
plt.ylabel('Count')
plt.title('Distribution of Test Preparation Courses')
plt.show()

*frequency of practicing sports*

In [ ]:
#frequency of practicing sports
plt.figure(figsize=(6,4))
df2['PracticeSport'].value_counts().plot(kind='bar')
plt.xlabel('Frequency of Practicing Sports')
plt.ylabel('Count')
plt.title('Distribution of Frequency of Practicing Sports')
plt.show()

In [ ]:
#firstchild distribution
plt.figure(figsize=(6,4))
df2['IsFirstChild'].value_counts().plot(kind='bar')
plt.xlabel('Is First Child?')
plt.ylabel('Count')
plt.title('First Child Distribution')
plt.show()

In [ ]:
#number of siblings
plt.figure(figsize=(6,4))
df2['NrSiblings'].hist(bins=8)
plt.xlabel('Number of Siblings')
plt.ylabel('Count')
plt.title('Distribution of Number of Siblings')
plt.show()

In [ ]:
#transportation means
plt.figure(figsize=(8,4))
df2['TransportMeans'].value_counts().plot(kind='bar')
plt.xlabel('Means of Transport to School')
plt.ylabel('Count')
plt.title('Transportation Distribution')
plt.show()

In [ ]:
#Study Hours Distribution
plt.figure(figsize=(6,4))
df2['WklyStudyHours'].hist(bins=20)
plt.xlabel('Weekly Self-Study Hours')
plt.ylabel('Count')
plt.title('Distribution of Weekly Self-Study Hours')
plt.show()

In [ ]:
#Math Scores
plt.figure(figsize=(6,4))
df2['MathScore'].hist(bins=20)
plt.xlabel('Math Score')
plt.ylabel('Count')
plt.title('Distribution of Math Test Scores')
plt.show()

In [ ]:
#Reading Scores
plt.figure(figsize=(6,4))
df2['ReadingScore'].hist(bins=20)
plt.xlabel('Reading Score')
plt.ylabel('Count')
plt.title('Distribution of Reading Test Scores')
plt.show()

In [ ]:
#Writing Scores
plt.figure(figsize=(6,4))
df2['WritingScore'].hist(bins=20)
plt.xlabel('Writing Score')
plt.ylabel('Count')
plt.title('Distribution of Writing Test Scores')
plt.show()

In [ ]:
#Scatterplot between Math and Reading
sns.scatterplot(x='MathScore', y='ReadingScore', data=df2)
plt.show()


In [ ]:
#Scatterplot between Math and Writing
sns.scatterplot(x='MathScore', y='WritingScore', data=df2)
plt.show()

In [ ]:
#Scatterplot between Reading and Writing
sns.scatterplot(x='ReadingScore', y='WritingScore', data=df2)
plt.show()

Note: Reading and Writing have a more narrow distribution

In [ ]:
#correlation map
corr = df2.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

Reading and Writing are highly correlated.
It doesn't seem (so far) that number of siblings is correlated much to any test


In [ ]:
#lets view the NaN values
#This will show how many NAN values exist on each feature
df2.isna().sum()

In [ ]:
# Another way to count NAN entries per feature.
# this should equal 1779
df2['TestPrep'].isna().sum()

**Preprocessing**

In [ ]:
#we can drop these values, but we may have significantly less data to use. 
#lets see the difference
df_dropped = df2.dropna().copy()

rows_df2 = df2.shape[0]
rows_df_dropped = df_dropped.shape[0]
print("Number of rows in df2:", rows_df2)
print("Number of rows in df_dropped:", rows_df_dropped)
print("Percentage of Rows that Remain:", rows_df_dropped / rows_df2)

In [ ]:
#Is this tradeoff worth it?
#For this sake of this analysis, I will remove the nulls and continue
NewDF = df_dropped.copy()
NewDF.head(10)

In [ ]:
#no remaining nulls


In [ ]:
NewDF.shape

In [ ]:
#Categorical Data Preprocessing

#-----------------------------------------#


#TestPrep: Label Encoding
#PracticeSport: Ordinal Encoding
#IsFirstChild: Label Encoding
#NrSiblings: Already Numerical
#TranportMeans: One-Hot Encoding
#WklyStudyHours: One-Hot Encoding

#-----------------------------------------#

In [ ]:
#TestPrep Unique
unique_testprep = NewDF['TestPrep'].unique()
print(unique_testprep)

In [ ]:
#TestPrep
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
NewDF['TestPrep_Encoded'] = label_encoder.fit_transform(NewDF['TestPrep'])
NewDF.drop('TestPrep', axis=1, inplace=True)
NewDF.head()

In [ ]:
#PracticeSport Unique Values
unique_ps = NewDF['PracticeSport'].unique()
print(unique_ps)

In [ ]:
#PracticeSport
#define the ordinal mapping dictionary
ordinal_mapping1 = {
    'never': 0,
    'sometimes': .5,
    'regularly': 1 
}
#apply ordinal encoding
NewDF['PracticeSport_Encoded'] = NewDF['PracticeSport'].map(ordinal_mapping1)
NewDF.drop('PracticeSport', axis=1, inplace=True)
NewDF.head()

In [ ]:
#IsFirstChild
from sklearn.preprocessing import LabelEncoder
label_encoder2 = LabelEncoder()
NewDF['IsFirstChild_Encoded'] = label_encoder2.fit_transform(NewDF['IsFirstChild'])
NewDF.drop('IsFirstChild', axis=1, inplace=True)
NewDF.head()

In [ ]:
#TransportMeans Unique
unique_TM = NewDF['TransportMeans'].unique()
print(unique_TM)

In [ ]:
#TransportMeans
from sklearn.preprocessing import LabelEncoder
label_encoder4 = LabelEncoder()
NewDF['TransportMeans_Encoded'] = label_encoder4.fit_transform(NewDF['TransportMeans'])
NewDF.drop('TransportMeans', axis=1, inplace=True)
NewDF.head()

In [ ]:
#WklyStudyHours
#PracticeSport
#define the ordinal mapping dictionary
ordinal_mapping3 = {
    '< 5': 0,
    '5 - 10': .5,
    '> 10': 1 
}
#apply ordinal encoding
NewDF['WklyStudyHours_Encoded'] = NewDF['WklyStudyHours'].map(ordinal_mapping3)
NewDF.drop('WklyStudyHours', axis=1, inplace=True)
NewDF.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
siblings = NewDF['NrSiblings'].values.reshape(-1, 1)
normalized_siblings = scaler.fit_transform(siblings)
NewDF['NrSiblings_normalized'] = normalized_siblings
NewDF.drop('NrSiblings', axis=1, inplace=True)
NewDF.head()

In [ ]:
# Remove missing values
NewDF.dropna()






# **Model Creation / Data Split**






In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#MathScore regression
X_math = NewDF.drop(["MathScore", "ReadingScore", "WritingScore"], axis=1)
y_math = NewDF["MathScore"]

#ReadingScore regression
X_reading = NewDF.drop(["MathScore", "ReadingScore", "WritingScore"], axis=1)
y_reading = NewDF["ReadingScore"]

#WritingScore regression
X_writing = NewDF.drop(["MathScore", "ReadingScore", "WritingScore"], axis=1)
y_writing = NewDF["WritingScore"]

X_math_train, X_math_test, y_math_train, y_math_test = train_test_split(X_math, y_math, test_size=0.2, random_state=42)
X_reading_train, X_reading_test, y_reading_train, y_reading_test = train_test_split(X_reading, y_reading, test_size=0.2, random_state=42)
X_writing_train, X_writing_test, y_writing_train, y_writing_test = train_test_split(X_writing, y_writing, test_size=0.2, random_state=42)

#MathScore regression
reg_math = LinearRegression()
reg_math.fit(X_math_train, y_math_train)

#ReadingScore regression
reg_reading = LinearRegression()
reg_reading.fit(X_reading_train, y_reading_train)

#WritingScore regression
reg_writing = LinearRegression()
reg_writing.fit(X_writing_train, y_writing_train)

#MathScore regression
y_math_pred = reg_math.predict(X_math_test)

#ReadingScore regression
y_reading_pred = reg_reading.predict(X_reading_test)

#WritingScore regression
y_writing_pred = reg_writing.predict(X_writing_test)


# **Model Summary and Analysis**

In [ ]:
# MathScore regression evaluation
mse_math = mean_squared_error(y_math_test, y_math_pred)
mae_math = mean_absolute_error(y_math_test, y_math_pred)
r2_math = r2_score(y_math_test, y_math_pred)

# ReadingScore regression evaluation
mse_reading = mean_squared_error(y_reading_test, y_reading_pred)
mae_reading = mean_absolute_error(y_reading_test, y_reading_pred)
r2_reading = r2_score(y_reading_test, y_reading_pred)

# WritingScore regression evaluation
mse_writing = mean_squared_error(y_writing_test, y_writing_pred)
mae_writing = mean_absolute_error(y_writing_test, y_writing_pred)
r2_writing = r2_score(y_writing_test, y_writing_pred)

# Print the evaluation metrics
print()
print("MathScore Regression:")
print("Mean Squared Error (MSE):", mse_math)
print("Mean Absolute Error (MAE):", mae_math)
print("R-squared (R2):", r2_math)
print()
print("ReadingScore Regression:")
print("Mean Squared Error (MSE):", mse_reading)
print("Mean Absolute Error (MAE):", mae_reading)
print("R-squared (R2):", r2_reading)
print()
print("WritingScore Regression:")
print("Mean Squared Error (MSE):", mse_writing)
print("Mean Absolute Error (MAE):", mae_writing)
print("R-squared (R2):", r2_writing)


In [ ]:
# MathScore regression feature importances
math_feature_importances = pd.DataFrame({'Feature': X_math.columns, 'Coefficient': reg_math.coef_})
math_feature_importances = math_feature_importances.sort_values(by='Coefficient', ascending=False)

# ReadingScore regression feature importances
reading_feature_importances = pd.DataFrame({'Feature': X_reading.columns, 'Coefficient': reg_reading.coef_})
reading_feature_importances = reading_feature_importances.sort_values(by='Coefficient', ascending=False)

# WritingScore regression feature importances
writing_feature_importances = pd.DataFrame({'Feature': X_writing.columns, 'Coefficient': reg_writing.coef_})
writing_feature_importances = writing_feature_importances.sort_values(by='Coefficient', ascending=False)

# Print feature importances for each model
print("MathScore Regression Feature Importances:")
print(math_feature_importances)
print()
print("ReadingScore Regression Feature Importances:")
print(reading_feature_importances)
print()
print("WritingScore Regression Feature Importances:")
print(writing_feature_importances)